In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys
from __future__ import division
#sys.path.append("/home/gias/dev/opinion/opinion/python/opinion")
sys.path.append(r"C:\dev\opinion\opinion\python\opinion")
import os
import utils.fileutils as fileutils
import utils.nlputils as nlputils
import cPickle as pickle
import pandas as pd
import numpy as np
import utils.metrics as metrics

from nltk import word_tokenize

In [2]:
indir = r"C:\dev\opinion\papers\sentimentDiversity"
dirDataset = os.path.join(indir, "dataset")

# Clean JIRA dataset

In [15]:
infile = os.path.join(dirDataset, "DatasetLinJIRA.csv")
outfile = os.path.join(dirDataset, "DatasetLinJIRA_Cleaned.csv")

dfi = pd.read_csv(infile)
sents = []
oracles = []
for index, row in dfi.iterrows():
    sent = row["sentence"]
    sent = sent.strip()
    sent = " ".join(sent.split('\n'))
    print "'%s'"%(sent)
    print "-----------------------------------"
    oracle = row["oracle"]
    sents.append(sent)
    oracles.append(oracle)
dfo = pd.DataFrame()
dfo["sentence"] = (pd.Series(sents)).values
dfo["oracle"] = (pd.Series(oracles)).values
dfo.to_csv(outfile, index=False)

'guys... this is so stupid...'
-----------------------------------
'I lost the whole morning cause HBase's RegionServer was dying with no logs, no nothing... how Am I supposed to debug the issue if u do not even generate a core dump?'
-----------------------------------
'{quote}You are messing down deep below hbase in dfs.'
-----------------------------------
'And I think if we're going to do a sweep up of shit, we should just kill root.'
-----------------------------------
'@idiot Yeah, I was on that idiot-path for a good while but commons-lang is a hairball, at least where this hashcode making and equals is concerned.'
-----------------------------------
'Pull it back in if you think different.'
-----------------------------------
'i suck.'
-----------------------------------
'Will still be stuck in the loop though if can't actually close regions.'
-----------------------------------
'Original edited patch was messed up... stupid sed!'
-----------------------------------
'Who made th

In [3]:
def map_score(score):
    if score > 0:
        return 'p'
    elif score < 0: 
        return 'n'
    else:
        return 'o'

In [4]:
def map_label(label):
    if label == "negative":
        return 'n'
    elif label == "positive":
        return 'p'
    else:
        return 'o'

In [17]:
## combine SentiSE labels 

def combineSentiSELabels(infileManual, outfileSentiSE, outfileResults):
    dfi = pd.read_csv(infileManual, sep=",",  encoding='ISO-8859-1')
    results = []
    with open(outfileSentiSE, 'rb') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.split("\t")
            out = parts[1].split()
            score = int(out[0]) + int(out[1])
            results.append(map_score(score)) 
    sents = []
    truths = []
    for index, row in dfi.iterrows():
        sents.append(row["sentence"])
        oracle = int(row["oracle"])
        truths.append(map_score(oracle))
    headers = ["Sentence", "ManualLabel", "SentiSE"]
    dfo = pd.DataFrame()
    dfo["Sentence"] = (pd.Series(sents)).values
    dfo["ManualLabel"] = (pd.Series(truths)).values
    dfo["SentiSE"] = (pd.Series(results)).values
    fileutils.writeExcel(outfileResults, "Sheet1", dfo)
    print "Done!"



In [ ]:
infileManual = os.path.join(dirDataset, "DatasetLinAppReviews.csv")
outfileSentiSE = os.path.join(dirDataset, "DatasetLinAppReviewsOutputSentiSE.txt")
outfileResults = os.path.join(dirDataset, "DatasetLinAppReviewsWithToolOutputs.xls")

combineSentiSELabels(infileManual, outfileSentiSE, outfileResults)

In [18]:
infileManual = os.path.join(dirDataset, "DatasetLinJIRA_Cleaned.csv")
outfileSentiSE = os.path.join(dirDataset, "DatasetLinJIRA_Cleaned_InputSentiSE_Output.txt")
outfileResults = os.path.join(dirDataset, "DatasetLinJIRAWithToolOutputs.xls")

combineSentiSELabels(infileManual, outfileSentiSE, outfileResults)

Done!


In [24]:
infileManual = os.path.join(dirDataset, "DatasetLinkStackOverflow.csv")
outfileSentiSE = os.path.join(dirDataset, "DatasetLinkStackOverflowInputSentiSE_Output.txt")
outfileResults = os.path.join(dirDataset, "DatasetLinkStackOverflowWithToolOutputs.xls")

combineSentiSELabels(infileManual, outfileSentiSE, outfileResults)

Done!


In [26]:
### combine sentise senti4sd labels

infileManual = os.path.join(dirDataset, "DatasetSenti4SD.csv")
dfi = pd.read_csv(infileManual)
sents = []
truths = []
for index, row in dfi.iterrows():
    sent = row["sentence"]
    #sent = sent.encode('ascii', 'ignore')
    #sent = unicode(sent)
    sents.append(sent)
    label = row["label"]
    truths.append(map_label(label))
outfileSentiSE = os.path.join(dirDataset, "DatasetSenti4SDOutputSentiSE.txt")
results = []
with open(outfileSentiSE, 'rb') as f:
    lines = f.readlines()
for line in lines:
    parts = line.split("\t")
    out = parts[1].split()
    score = int(out[0]) + int(out[1])
    results.append(map_score(score)) 
outfileResults = os.path.join(dirDataset, "DatasetSenti4SDWithToolOutputs.csv")
headers = ["Sentence", "ManualLabel", "SentiSE"]
dfo = pd.DataFrame()
dfo["Sentence"] = (pd.Series(sents)).values
dfo["ManualLabel"] = (pd.Series(truths)).values
dfo["SentiSE"] = (pd.Series(results)).values
dfo.to_csv(outfileResults, columns = headers)
#fileutilsl(outfileResults, "Sheet1", dfo)
print "Done!"

Done!


# Prepare Datasets for Senti4SD

In [8]:
def consolidateToolOutput(infile, outfile):
    dfi = pd.read_csv(infile)
    results = dict()
    for index, row in dfi.iterrows():
        rowNum = row["Row"]
        label = row["Predicted"]
        label = map_label(label)
        rowNum = rowNum.replace('t', '')
        rowNum = int(rowNum)
        rowNum = rowNum
        results[rowNum] = label
    outs = []
    dfo = fileutils.readExcel(outfile, 'Sheet1')
    for index, row in dfo.iterrows():
        outs.append(results[index])
    dfo["Senti4SD"] = (pd.Series(outs)).values
    fileutils.writeExcel(outfile, "Sheet1", dfo)

In [20]:
infile = "DatasetLinAppReviewsInputSenti4SD_Output.csv"
infile = os.path.join(dirDataset, infile)

outfile = "DatasetLinAppReviewsWithToolOutputs.xls"
outfile = os.path.join(dirDataset, outfile)

consolidateToolOutput(infile, outfile)

In [22]:
infile = "DatasetLinJIRA_Cleaned_InputSenti4SD_Output.csv"
infile = os.path.join(dirDataset, infile)

outfile = "DatasetLinJIRAWithToolOutputs.xls"
outfile = os.path.join(dirDataset, outfile)

consolidateToolOutput(infile, outfile)

In [25]:
infile = "DatasetLinkStackOverflowInputSenti4SD_Output.csv"
infile = os.path.join(dirDataset, infile)

outfile = "DatasetLinkStackOverflowWithToolOutputs.xls"
outfile = os.path.join(dirDataset, outfile)

consolidateToolOutput(infile, outfile)

# Prepare Ortu Dataset

In [3]:
infileManual = os.path.join(dirDataset, "Ortu_all_output_SentiSE.csv")
outfileResults = os.path.join(dirDataset, "DatasetOrtuJiraWithToolOutputs.xls")

In [8]:
dfi = pd.read_csv(infileManual, encoding='ISO-8859-1')
dfi.head()

,id,label,comment,score
0,180,neutral,Jimmy this ready for review/commit? If so plea...,2 -1
1,409,neutral,I'm backporting this to 3.4 and 3.3 branches. ...,1 -1
2,485,neutral,+1 overall. Here are the results of testing th...,1 -1
3,611,negative,Bug Flavio more? Seriously,1 -1
4,644,neutral,Integrated in ZooKeeper-trunk #1266 (See https...,1 -1


In [9]:
headers = ["sentence", "ManualLabel", "SentiSE"]
data = []
for index, row in dfi.iterrows():
    label = row["label"]
    if label == "neutral":
        label = 'o'
    elif label == "positive":
        label = 'p'
    elif label == "negative":
        label = 'n'
    else:
        print "wtf ", label
    score = row["score"].split()
    score = int(score[0]) + int(score[1])
    if score > 0:
        got = 'p'
    elif score < 0:
        got = 'n'
    else:
        got = 'o'
    record = {"sentence": row["comment "], "ManualLabel": label, "SentiSE": got}
    data.append(record)
dfo = pd.DataFrame.from_dict(data)
fileutils.writeExcel(outfileResults, "Sheet1", dfo)

In [10]:
# add senti4sd results
infiles4Res = os.path.join(dirDataset, "Ortu_all_Input_Senti4SD_pred.csv")
dfs = pd.read_csv(infiles4Res)
dfs.head()

,Row,Predicted
0,t3,neutral
1,t4,neutral
2,t2,positive
3,t7,neutral
4,t5,neutral


In [11]:
res = dict()
for index, row in dfs.iterrows():
    rid = row["Row"]
    rid = int(rid.split('t')[1])
    label = row["Predicted"]
    if label == "neutral":
        label = 'o'
    elif label == "positive":
        label = 'p'
    elif label == "negative":
        label = 'n'
    else:
        print "wtf ", label
    res[rid] = label

In [12]:
s4res = []
for index, row in dfo.iterrows():
    s4res.append(res[index])
dfo["Senti4SD"] = (pd.Series(s4res)).values
fileutils.writeExcel(outfileResults, "Sheet1", dfo)